In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
data = pd.read_csv('data/hurdat2-formattted.csv', sep = ";")
data.head()

,Identifier,Name,Date,Time,Land,System Status,Longitude,Latitude,Max wind,Min pressure,34 kt wind northeastern,34 kt wind radii southeastern,34 kt wind radii southwestern,34 kt wind radii southwestern.1,50 kt wind northeastern,50 kt wind radii southeastern,50 kt wind radii southwestern,50 kt wind radii southwestern.1,64 kt wind northeastern,64 kt wind radii southeastern,64 kt wind radii southwestern,64 kt wind radii southwestern.1
0,AL011851,UNNAMED,18510625,0,0,HU,-94.8,28.0,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,0,HU,-95.4,28.0,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,0,HU,-96.0,28.0,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,0,HU,-96.5,28.1,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,1,HU,-96.8,28.2,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [10]:
# Data cleaning
columns_to_drop = ['Name', 'Date', 'Time']

for column in columns_to_drop:
    if column in data.columns:
        data.drop(column, axis = 1, inplace = True)
        print("Colum '{}' dropped".format(column))
    else:
        print("Column '{}' already has been dropped".format(column))

Colum 'Name' dropped
Colum 'Date' dropped
Colum 'Time' dropped


In [0]:
# One Hot Encoding: System Status column
data = pd.get_dummies(data, columns = ['System Status'])

In [12]:
data

,Identifier,Land,Longitude,Latitude,Max wind,Min pressure,34 kt wind northeastern,34 kt wind radii southeastern,34 kt wind radii southwestern,34 kt wind radii southwestern.1,50 kt wind northeastern,50 kt wind radii southeastern,50 kt wind radii southwestern,50 kt wind radii southwestern.1,64 kt wind northeastern,64 kt wind radii southeastern,64 kt wind radii southwestern,64 kt wind radii southwestern.1,System Status_DB,System Status_ET,System Status_EX,System Status_HU,System Status_LO,System Status_SD,System Status_SS,System Status_TD,System Status_TS,System Status_WV
0,AL011851,0,-94.8,28.0,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,0,0,1,0,0,0,0,0,0
1,AL011851,0,-95.4,28.0,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,0,0,1,0,0,0,0,0,0
2,AL011851,0,-96.0,28.0,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,0,0,1,0,0,0,0,0,0
3,AL011851,0,-96.5,28.1,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,0,0,1,0,0,0,0,0,0
4,AL011851,1,-96.8,28.2,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51341,AL162018,1,-19.6,57.9,55,960,780,600,660,480,0,240,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
51342,AL162018,1,-17.1,58.9,50,964,480,600,660,420,0,240,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
51343,AL162018,1,-14.5,59.8,45,968,360,480,480,360,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
51344,AL162018,1,-12.1,60.8,40,973,270,330,360,270,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [13]:
# Version a.1: Training whole dataset
y = data['Land']
X = data.drop(['Land', 'Identifier'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size    = 0.33,
    random_state = 42
)

rf_a = RandomForestClassifier(
    random_state = 17,
    verbose = 3
)

rf_a.fit(X_train, y_train)

y_prediction = rf_a.predict(X_test)

mse = mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)

print("Mean squared error: {}".format(rmse))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Mean squared error: 0.1545990158754216


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


In [14]:
# Version a.2 with cross validation
def display_scores(scores):
    print("Scores: {}".format(scores))
    print("Mean: {}".format(scores.mean()))
    print("Standard deviation: {}".format(scores))

rf_a_cross_val = RandomForestClassifier(
    random_state = 17,
    verbose = 0
)
scores = cross_val_score(
    rf_a_cross_val,
    X_train,
    y_train,
    scoring="neg_mean_squared_error",
    cv=10
)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)

Scores: [0.15716909 0.16084804 0.16442288 0.15626453 0.17303716 0.15154246
 0.1635365  0.15903049 0.14366466 0.16084804]
Mean: 0.15903638608669943
Standard deviation: [0.15716909 0.16084804 0.16442288 0.15626453 0.17303716 0.15154246
 0.1635365  0.15903049 0.14366466 0.16084804]


In [15]:
# Version a.3 with Grid Search and cross validation
param_grid = [
    {
        'n_estimators': [3, 10, 30],
        'max_features': [4, 6, 8, 15]
    },
    {
        'bootstrap': [False],
        'n_estimators': [3, 10],
        'max_features': [2, 3, 4]
    }
]

rf_a_grid_search = RandomForestClassifier(
    random_state = 17,
    verbose = 0
)

grid_search = GridSearchCV(
    rf_a_grid_search,
    param_grid,
    cv=10,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

grid_search.fit(X_train, y_train)
cvres = grid_search.cv_results_

print("Best params: {}".format(grid_search.best_params_))

for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

Best params: {'max_features': 15, 'n_estimators': 30}
0.18893553214648512 {'max_features': 4, 'n_estimators': 3}
0.17058148519653019 {'max_features': 4, 'n_estimators': 10}
0.16547805277193103 {'max_features': 4, 'n_estimators': 30}
0.17702020589492187 {'max_features': 6, 'n_estimators': 3}
0.16057443364336577 {'max_features': 6, 'n_estimators': 10}
0.1537302375890638 {'max_features': 6, 'n_estimators': 30}
0.16946988873092556 {'max_features': 8, 'n_estimators': 3}
0.1512518472857567 {'max_features': 8, 'n_estimators': 10}
0.14234123820896583 {'max_features': 8, 'n_estimators': 30}
0.12563625975843984 {'max_features': 15, 'n_estimators': 3}
0.11824640277470815 {'max_features': 15, 'n_estimators': 10}
0.11296599191275314 {'max_features': 15, 'n_estimators': 30}
0.19304501393499418 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
0.17954793149118572 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
0.18591153873420038 {'bootstrap': False, 'max_features': 3, 'n_estimat

In [16]:
feature_importances = grid_search.best_estimator_.feature_importances_
sorted(zip(feature_importances, X_train.columns), reverse=True)

[(0.5171196841217689, 'Longitude'),
 (0.4020216540178943, 'Latitude'),
 (0.03576535964136791, 'Max wind'),
 (0.019603743156869344, 'System Status_EX'),
 (0.009978619711947662, 'Min pressure'),
 (0.003036436326491529, 'System Status_TD'),
 (0.0016492995219277696, 'System Status_HU'),
 (0.0016331934235584405, 'System Status_TS'),
 (0.001455925047990984, '34 kt wind radii southwestern.1'),
 (0.0014438704977330443, '34 kt wind radii southeastern'),
 (0.0011211371470319636, '34 kt wind northeastern'),
 (0.0007916736761781581, '34 kt wind radii southwestern'),
 (0.0006728212258034022, '50 kt wind radii southeastern'),
 (0.0005520248638804644, '50 kt wind northeastern'),
 (0.0005067094472769474, '50 kt wind radii southwestern.1'),
 (0.0004461358865129061, 'System Status_LO'),
 (0.0003889152052690619, '64 kt wind northeastern'),
 (0.0003750649512804725, '50 kt wind radii southwestern'),
 (0.00036571879969390394, '64 kt wind radii southwestern'),
 (0.00032684665783953913, '64 kt wind radii sout

In [17]:
final_model = grid_search.best_estimator_

final_predictions = final_model.predict(X_test)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print("Final rmse: {}".format(final_rmse))

Final rmse: 0.1030660105836144


In [18]:
# Create array with hurricanes
hurricanes = {}
hurricane_identifiers = data['Identifier'].unique()

for hurricane_id in tqdm(hurricane_identifiers):
    hurricanes[hurricane_id] = data[data['Identifier'] == hurricane_id].drop('Identifier', axis = 1)

100%|██████████| 1873/1873 [00:08<00:00, 226.02it/s]


In [19]:
# Now, you can predict hurricanes!
hurricane_to_predict = hurricanes['AL162018']

hurricane_data_to_predict = hurricane_to_predict.drop('Land', axis = 1)

hurricane_row_to_predict = hurricane_data_to_predict[15:16]
print("Prediction: {} vs fact: {}".format(final_model.predict(hurricane_row_to_predict), hurricane_to_predict[15:16]['Land'].values))

hurricane_row_to_predict = hurricane_data_to_predict[20:21]
print("Prediction: {} vs fact: {}".format(final_model.predict(hurricane_row_to_predict), hurricane_to_predict[20:21]['Land'].values))

hurricane_row_to_predict = hurricane_data_to_predict[25:26]
print("Prediction: {} vs fact: {}".format(final_model.predict(hurricane_row_to_predict), hurricane_to_predict[25:26]['Land'].values))


Prediction: [0] vs fact: [0]
Prediction: [0] vs fact: [0]
Prediction: [1] vs fact: [1]
